In [17]:
!nvidia-smi

Tue Jun  7 18:25:02 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.142.00   Driver Version: 450.142.00   CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            On   | 00000000:00:1E.0 Off |                    0 |
| N/A   42C    P0    26W /  70W |  15103MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Settings

In [4]:
import warnings
warnings.filterwarnings('ignore')

from glob import glob
import pandas as pd
import numpy as np 
from tqdm import tqdm
import cv2

import os
import timm
import random

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torchvision.transforms as transforms
from sklearn.metrics import f1_score, accuracy_score
import time

from sklearn.model_selection import StratifiedKFold
device = torch.device('cuda')

## Image Pre-Process

In [6]:
train_png = sorted(glob('/home/lab17/jupyter_home/Data/bdata_snacks/*.jpg'))
test_png = sorted(glob('/home/lab17/jupyter_home/git/git_jy/test_dataset/*.jpg'))

# files = os.listdir('/home/lab17/jupyter_home/Data/bdata_snacks')
# files

In [7]:
print(len(train_png), len(set(train_png)))
print(len(test_png), len(set(test_png)))

311 311
7 7


In [8]:
# decoder
files = os.listdir('/home/lab17/jupyter_home/Data/bdata_snacks')
train_labels = sorted(files)
train_dict = {i:l[:-4] for i, l in zip(range(311), train_labels)}

In [9]:
def img_load(path):
    img = cv2.imread(path)[:,:,::-1]
    img = cv2.resize(img, (384, 384),interpolation = cv2.INTER_AREA)
    return img

In [31]:
train_imgs = [img_load(m) for m in tqdm(train_png)]
test_imgs = [img_load(n) for n in tqdm(test_png)]

100%|████████████████████████████████████████████| 7/7 [00:00<00:00, 131.86it/s]


In [32]:
np.save('/home/lab17/jupyter_home/Data/bdata_snacks_384.npy', np.array(train_imgs))
np.save('/home/lab17/jupyter_home/Data/test_snacks_384.npy', np.array(test_imgs))

In [10]:
train_imgs = np.load('/home/lab17/jupyter_home/Data/bdata_snacks_384.npy')
test_imgs = np.load('/home/lab17/jupyter_home/Data/test_snacks_384.npy')

In [13]:
# meanRGB = [np.mean(x, axis=(0,1)) for x in train_imgs]
# stdRGB = [np.std(x, axis=(0,1)) for x in train_imgs]

# meanR = np.mean([m[0] for m in meanRGB])/255
# meanG = np.mean([m[1] for m in meanRGB])/255
# meanB = np.mean([m[2] for m in meanRGB])/255

# stdR = np.mean([s[0] for s in stdRGB])/255
# stdG = np.mean([s[1] for s in stdRGB])/255
# stdB = np.mean([s[2] for s in stdRGB])/255

# print("train 평균",meanR, meanG, meanB)
# print("train 표준편차",stdR, stdG, stdB)

train 평균 0.7097844470485838 0.633531969760679 0.5547763538712672
train 표준편차 0.2033680463341466 0.21580010480417353 0.24844798400637713


In [34]:
# meanRGB = [np.mean(x, axis=(0,1)) for x in test_imgs]
# stdRGB = [np.std(x, axis=(0,1)) for x in test_imgs]

# meanR = np.mean([m[0] for m in meanRGB])/255
# meanG = np.mean([m[1] for m in meanRGB])/255
# meanB = np.mean([m[2] for m in meanRGB])/255

# stdR = np.mean([s[0] for s in stdRGB])/255
# stdG = np.mean([s[1] for s in stdRGB])/255
# stdB = np.mean([s[2] for s in stdRGB])/255

# print("test 평균",meanR, meanG, meanB)
# print("test 표준편차",stdR, stdG, stdB)

test 평균 0.5675011557357318 0.5264908952947499 0.42253120486475837
test 표준편차 0.2274250397086117 0.23777227831017764 0.25968101419272444


## Model Define

In [12]:
class Custom_dataset(Dataset):
    def __init__(self, img_paths, labels, mode='train'):
        self.img_paths = img_paths
        self.labels = labels
        self.mode=mode
    def __len__(self):
        return len(self.img_paths)
    def __getitem__(self, idx):
        img = self.img_paths[idx]
        if self.mode == 'train':
#             train_transform = transforms.Compose([
#                 transforms.ToTensor(),
#                 transforms.Normalize(mean = [0.709784, 0.633531, 0.554776],
#                                     std = [0.203368, 0.215800, 0.248448]),
#                 transforms.RandomCrop(224),
#                 transforms.RandomPerspective(),
#                 transforms.RandomAffine((20)),  # x, y축으로 이미지 늘림
#                 transforms.RandomRotation(90),
#                 transforms.ColorJitter(brightness=0.5),
#             ])


            train_transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean = [0.709784, 0.633531, 0.554776],
                                    std = [0.203368, 0.215800, 0.248448]),
                transforms.RandomAffine((-45, 45)),
                transforms.RandomRotation((0,80))       #  이미지를 랜덤으로 degrees 각도로 회전한다.
            ])
            img = train_transform(img)
        if self.mode == 'test':
            test_transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean = [0.567501, 0.526491, 0.422531],
                                     std = [0.227425, 0.237772, 0.259681]),

            ])
            img = test_transform(img)

        
        label = self.labels[idx]
        return img, label
    
class Network(nn.Module):
    def __init__(self,mode = 'train'):
        super(Network, self).__init__()
        self.mode = mode
        if self.mode == 'train':
          self.model = timm.create_model('efficientnet_b4', pretrained=True, num_classes=311, drop_path_rate = 0.2)
        if self.mode == 'test':
          self.model = timm.create_model('efficientnet_b4', pretrained=True, num_classes=311, drop_path_rate = 0)
        
    def forward(self, x):
        x = self.model(x)
        return x

In [13]:
def score_function(real, pred):
#     score = f1_score(real, pred, average="macro")
    score = accuracy_score(real, pred)
    return score

In [14]:
def main(seed = 2022):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = True
    
main(2022)

## Train

In [16]:
import gc

batch_size = 34
epochs = 70
pred_ensemble = []

# Train
train_dataset = Custom_dataset(np.array(train_imgs), np.array(list(train_dict.keys())), mode='train')
train_loader = DataLoader(train_dataset, shuffle=False, batch_size=batch_size)

# # Val
# val_dataset = Custom_dataset(np.array(val_imgs), np.array(val_labels), np.array([]), mode='test')
# val_loader = DataLoader(val_dataset, shuffle=True, batch_size=batch_size)


gc.collect()
torch.cuda.empty_cache()
best=0

model = Network().to(device)

# optimizer = torch.optim.AdamW(model.parameters(), lr=2e-4, weight_decay = 2e-2)
optimizer = torch.optim.Adadelta(model.parameters())
criterion = nn.CrossEntropyLoss()
scaler = torch.cuda.amp.GradScaler()  

best_f1 = 0
early_stopping = 0
for epoch in range(epochs):
    start=time.time()
    train_loss = 0
    train_pred=[]
    train_y=[]
    model.train()
    for batch in (train_loader):
        optimizer.zero_grad()
        x = torch.tensor(batch[0], dtype=torch.float32, device=device)
        y = torch.tensor(batch[1], dtype=torch.long, device=device)
        with torch.cuda.amp.autocast():
            pred = model(x)
        loss = criterion(pred, y)


        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        train_loss += loss.item()/len(train_loader)
        train_pred += pred.argmax(1).detach().cpu().numpy().tolist()
        train_y += y.detach().cpu().numpy().tolist()
        
    train_f1 = score_function(train_y, train_pred)
#     state_dict= model.state_dict()
#     model.eval()


    TIME = time.time() - start
    print(f'epoch : {epoch+1}/{epochs}    time : {TIME:.0f}s/{TIME*(epochs-epoch-1):.0f}s')
    print(f'TRAIN    loss : {train_loss:.5f}    acc : {train_f1:.5f}')
#     print(f'Val    loss : {val_loss:.5f}    f1 : {val_f1:.5f}')

RuntimeError: CUDA out of memory. Tried to allocate 26.00 MiB (GPU 0; 14.76 GiB total capacity; 10.77 GiB already allocated; 6.75 MiB free; 11.11 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

## Save

In [56]:
torch.save({'epoch':epoch,
            'state_dict':state_dict,
            'optimizer': optimizer.state_dict(),
            'scaler': scaler.state_dict(),
     }, '/home/lab17/jupyter_home/Model/snacks_311_2.pth')


## Predict

In [57]:
pred_ensemble = []
batch_size = 34
# Test
test_dataset = Custom_dataset(np.array(test_imgs), np.array(["tmp"]*len(test_imgs)), mode='test')
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=batch_size)

model_test = Network(mode = 'test').to(device)
model_test.load_state_dict(torch.load('/home/lab17/jupyter_home/Model/snacks_311_2.pth')['state_dict'])
model_test.eval()
f_pred = []
pred_prob = []

with torch.no_grad():
    for batch in (test_loader):
        x = torch.tensor(batch[0], dtype = torch.float32, device = device)
        with torch.cuda.amp.autocast():
            pred = model_test(x)
            pred_prob.extend(pred.detach().cpu().numpy())
        f_pred.extend(pred.argmax(1).detach().cpu().numpy().tolist())

In [58]:
# label_decoder = {val:key for key, val in label_unique.items()}

f_result = [train_dict[result] for result in f_pred]

In [59]:
f_result

['단백질 오란다 100g',
 '유기농 쌀떡뻥 25g',
 '(G) 노브랜드쿠키앤크림샌드720g',
 '(G) 노브랜드쿠키앤크림샌드720g',
 '(G) 노브랜드쿠키앤크림샌드720g',
 '(G) 노브랜드쿠키앤크림샌드720g',
 '(G) 노브랜드쿠키앤크림샌드720g']

##### 정답
- 감자깡
- 알새우칩
- 포스틱
- 사또밥
- 고래밥
- 빠삭칩
- 칸츄리콘

##### 1번째 : snacks_311

- '[농심] 새우깡90g'
- '[농심] 알새우칩 68g',
- '[농심] 포스틱 84g',
- '오리온 치즈뿌린 치킨팝 81g',
- '맘스케어 유기농까까 오리지날 43g',
- '노브랜드 달콤한꿀깨맛꽈배기 200 g',
- '노브랜드 땅콩카라멜콘 230 g'

##### 2번째 : snacks_311_2